# GloVe Assignment

<b>Name:</b> Pyae Sone Kyaw <b>Student Id:</b> st123225 <b>Course:</b> CS NLU <b>Assignment_2:</b> Glove Assignment <b>Submission Date:</b> 26/1/2023

inspired by: Abhniav's and Prof Chalky's code

## Introduction


## Implementation Details
use all the models we have learned so far. (skip-gram, skipgram with negative sampling, CBOw and Glove)
## assessment
use word analogy task dataset to assess the models. (syntactic and semantic accuracy)
## corpus 
use brown corpus. (news category) since it is a large corpus. most of my classmates used it.
## preprocessing
filter out stopwords and punctuations. (to reduce the vocab size)
## vocab size
use the first 2000 sentences from the corpus category. (to reduce the vocab size)





## similarity dataset
use wordsim353_sim_rel dataset. (to check the correlation of the similarity given by the trained models with the similarity score given by the judge for word pairs similarity)
## correlation metric
use Spearman correlation coefficient with associated p-value. (to check the correlation of the similarity given by the trained models with the similarity score given by the judge for word pairs similarity)





In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

## 1) Load Data

In [2]:
#let's load the data and remove all the punctuations and stop words

In [3]:
import nltk

In [4]:
nltk.download('brown')

[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\pyaes\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

In [5]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\pyaes\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import string

In [7]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
punctuations = []
for i in string.punctuation:
    punctuations.append(i)
punctuations.append("''")
punctuations.append('""')
punctuations.append('``')

In [9]:
from nltk.corpus import stopwords

In [10]:
stop_words = set(stopwords.words('english'))

In [11]:
from nltk.corpus import brown

In [12]:
brown.categories()

['adventure',
 'belles_lettres',
 'editorial',
 'fiction',
 'government',
 'hobbies',
 'humor',
 'learned',
 'lore',
 'mystery',
 'news',
 'religion',
 'reviews',
 'romance',
 'science_fiction']

In [13]:
#we list all the stopwords and punctuations above and we are gonna remove them from the corpus
corpus_tokenized = brown.sents(categories='news')

corpus_tokenized[0]

corpus_tokenized = [[word.lower() for word in sent if word not in punctuations and word not in stop_words] for sent in corpus_tokenized]

corpus_tokenized[0]



['the',
 'fulton',
 'county',
 'grand',
 'jury',
 'said',
 'friday',
 'investigation',
 "atlanta's",
 'recent',
 'primary',
 'election',
 'produced',
 'evidence',
 'irregularities',
 'took',
 'place']

In [14]:
len(corpus_tokenized)

4623

In [15]:
corpus_tokenized = corpus_tokenized[:2000]

## 2) Prepare Data for Training

### Creating vocabulary

In [16]:
#we want to flatten this (basically merge all list)
flatten = lambda l: [item for sublist in l for item in sublist]
vocabs  = list(set(flatten(corpus_tokenized)))  #vocabs is a term defining all unique words your system know

In [17]:
print(vocabs[:10])

['comforting', 'consequently', 'streamlined', 'deadline', 'chesapeake', 'nursing', 'bar', 'principal', 'thrill', 'bankers']


In [18]:
print("Vocabulary size:", len(vocabs))

Vocabulary size: 7323


### Numericalization

In [19]:
word2index = {v: idx for idx, v in enumerate(vocabs)}

In [20]:
#add <UNK>, which is a very normal token exists in the world
vocabs.append('<UNK>') 

In [21]:
#now we have a way to know what is the id of <UNK>
word2index['<UNK>'] = 6  #usually <UNK> is 0

In [22]:
#create index2word dictionary
#2 min    
index2word = {v:k for k, v in word2index.items()}

In [23]:
word2index["place"]

5547

In [24]:
#print one index2word
index2word[0]
#print another index2word larger 
index2word[1000]

'bombs'

### let's create context and outside words for training the model using skip-gram , skip-gram with negative sampling and golve as instructed in the assignment


In [25]:
window_size = 2

In [26]:
skipgrams = []

#for each corpus
for sent in corpus_tokenized:
    #for each sent
    for i in range(window_size, len(sent) - window_size):
        center_word = word2index[sent[i]]
        outside_words = []
        for j in range(1, window_size + 1):
            outside_words.append(word2index[sent[i-j]])
            outside_words.append(word2index[sent[i+j]])
            
        for o in outside_words:
            skipgrams.append([center_word, o])

skipgrams[:10]

[[4813, 523],
 [4813, 7301],
 [4813, 3108],
 [4813, 2851],
 [7301, 4813],
 [7301, 2851],
 [7301, 523],
 [7301, 7229],
 [2851, 7301],
 [2851, 7229]]

### Unigram distribution
$$P(w)=U(w)^{3/4}/Z$$

Defining the probability of sampling negative words

In [27]:
from collections import Counter

In [28]:
z = 0.001

In [29]:
word_count = Counter(flatten(corpus_tokenized))

In [30]:
num_total_words = sum([c for w, c in word_count.items()])
num_total_words

23735

In [31]:
unigram_table = []

for v in vocabs:
    uw = word_count[v]/num_total_words
    uw_alpha = uw ** 0.75
    uw_alpha_dividebyz = int(uw_alpha / z)
    # print("vocab: ", v)
    # print("distribution: ", uw_alpha_dividebyz)
    unigram_table.extend([v] * uw_alpha_dividebyz)

In [32]:
unigram_table[:5]

['nursing', 'nursing', 'bar', 'principal', 'bankers']

### now the pair for context and outside words are ready for training the model using skip-gram , skip-gram with negative sampling and golve as instructed in the assignment . let's move on to CBOW

In [33]:
skipgrams_CBOW = []

#for each corpus
for sent in corpus_tokenized:
    for i in range(window_size, len(sent) - window_size):
        center_word   = word2index[sent[i]]
        outside_words = []
        
        low  = i - window_size
        high = i + window_size
        for j in range(low, high + 1):
            if j == i:
                continue
            outside_words.append(word2index[sent[j]])
        skipgrams_CBOW.append([center_word, outside_words])

skipgrams_CBOW[:10]

[[4813, [3108, 523, 7301, 2851]],
 [7301, [523, 4813, 2851, 7229]],
 [2851, [4813, 7301, 7229, 2789]],
 [7229, [7301, 2851, 2789, 1154]],
 [2789, [2851, 7229, 1154, 6003]],
 [1154, [7229, 2789, 6003, 2002]],
 [6003, [2789, 1154, 2002, 7258]],
 [2002, [1154, 6003, 7258, 4313]],
 [7258, [6003, 2002, 4313, 5583]],
 [4313, [2002, 7258, 5583, 895]]]

### Co-occurrence matrix

In [34]:
#we want to flatten this (basically merge all list)
    #vocabs is a term defining all unique words your system know
X_i = Counter(flatten(corpus_tokenized)) 

In [35]:
gloVes = []

#loop through each corpus
for sent in corpus_tokenized: 
    #loop through each word from 1 to n-1 (because 0 and n has no context window)
    for i in range(1, len(sent)-1):
        target  = sent[i]
        context = [sent[i+1], sent[i-1]]
        #append(i, i+1) and append(i, i-1)
        for c in context:
            gloVes.append((target, c))

In [36]:
gloVe_id = [(word2index[gloVe[0]], word2index[gloVe[1]]) for gloVe in gloVes]

In [37]:
print(gloVe_id[:5])

[(523, 4813), (523, 3108), (4813, 7301), (4813, 523), (7301, 2851)]


In [38]:
#since we have these occurrences, we can count, to make our co-occurrence matrix!!!
X_ik_skipgram = Counter(gloVes)

In [39]:
X_ik_skipgram[("fulton", "county")]

6

### Weighting function f

In [40]:
def weighting(w_i, w_j, X_ik): #X_ik is the co-occurrence matrix
    #define x_ij
    try:
        x_ij = X_ik[(w_i, w_j)]
    except:
        x_ij = 1  #why one, so that the probability thingy won't break...(label smoothing)
        
    #define x_max and alpha
    x_max = 100
    alpha = 0.75
    
    #define the weighting function
    if x_ij < x_max:
        result = (x_ij/x_max) ** alpha
    else:
        result = 1 #if it's too big, we just set it to 1
        
    return result


In [41]:
w_i  = 'fulton'
w_j  = 'state'
w_j2 = 'chaky'

print(weighting(w_i, w_j, X_ik_skipgram)) #this is the probability of co-occurrence between fulton and state
print(weighting(w_i, w_j2, X_ik_skipgram)) #this is the probability of co-occurrence between fulton and chaky

0.0
0.0


In [42]:
#now apply this weighting to alcorpus_tokenizedible pairs
from itertools import combinations_with_replacement

X_ik = {} #for keeping the co-occurrences
weighting_dic = {} #for keeping all the probability after passing through the weighting function

for bigram in combinations_with_replacement(vocabs, 2):  #we need to also think its reverse
    #if this bigram exists in X_ik_skipgrams
    #we gonna add this to our co-occurence matrix
    if X_ik_skipgram.get(bigram) is not None:
        cooc = X_ik_skipgram[bigram]  #get the co-occurrence
        X_ik[bigram] = cooc + 1 #this is again basically label smoothing....(stability issues (especially when divide something))
        X_ik[(bigram[1], bigram[0])] = cooc + 1  #trick to get all pairs
    else: #otherwise, do nothing
        pass
    
    #apply the weighting function using this co-occurrence matrix thingy    
    weighting_dic[bigram] = weighting(bigram[0], bigram[1], X_ik)
    weighting_dic[(bigram[1], bigram[0])] = weighting(bigram[1], bigram[0], X_ik)

In [43]:
len(X_ik_skipgram)

36226

### create random vectors for the words in the vocabulary

In [44]:
def random_batch(batch_size, corpus, skip_grams):
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(skip_grams)), batch_size, replace=False) #randomly pick without replacement
        
    for i in random_index:
        random_inputs.append([skip_grams[i][0]])  # target, e.g., 2
        random_labels.append([skip_grams[i][1]])  # context word, e.g., 3
            
    return np.array(random_inputs), np.array(random_labels)

<b>Test Function</b>

In [45]:
input, label = random_batch(5, corpus_tokenized, skipgrams)

print("Input sample:", input)
print(f"Input shape: {input.shape}", end="\n\n")
print(f"Label sample: {label=}")
print("Label shape:", label.shape)

Input sample: [[   9]
 [4737]
 [ 122]
 [4435]
 [6087]]
Input shape: (5, 1)

Label sample: label=array([[6528],
       [5294],
       [1556],
       [1043],
       [4891]])
Label shape: (5, 1)


In [46]:
#putting it all together in a function for CBOW
def random_batch_cbow(batch_size, corpus, cbow):
    random_inputs = []
    random_labels = []
    random_index = np.random.choice(range(len(cbow)), batch_size, replace=False) #randomly pick without replacement
        
    for i in random_index:
        random_inputs.append([cbow[i][0]])  # target, e.g., 2
        random_labels.append([cbow[i][1]])  # context word, e.g., 3
            
    return np.array(random_inputs), np.array(random_labels).squeeze()

<b>Test Function</b>

In [47]:
input, label = random_batch_cbow(5, corpus_tokenized, skipgrams_CBOW)

print("Input sample:", input)
print(f"Input shape: {input.shape}", end="\n\n")
print(f"Label sample: {label=}")
print("Label shape:", label.shape)

Input sample: [[4313]
 [3813]
 [5514]
 [2864]
 [5621]]
Input shape: (5, 1)

Label sample: label=array([[2195, 2233, 4533, 1851],
       [7171, 6596, 2011, 3620],
       [1111, 3512, 2103, 2825],
       [7229, 4533, 2711, 4273],
       [ 510, 3494, 1006, 6999]])
Label shape: (5, 4)


In [48]:
import math

def random_batch_gloVe(batch_size, word_sequence, skip_grams, X_ik, weighting_dic):
    
    #loop through this skipgram, and change it id  because when sending model, it must number
    skip_grams_id = [(word2index[skip_gram[0]], word2index[skip_gram[1]]) for skip_gram in skip_grams]
    
    #randomly pick "batch_size" indexes
    number_of_choices = len(skip_grams_id)
    random_index = np.random.choice(number_of_choices, batch_size, replace=False) #no repeating indexes among these random indexes
    
    random_inputs = [] #xi, wi (in batches)
    random_labels = [] #xj, wj (in batches)
    random_coocs  = [] #Xij (in batches)
    random_weighting = [] #f(Xij) (in batches)
    #for each of the sample in these indexes
    for i in random_index:
        random_inputs.append([skip_grams_id[i][0]]) #same reason why i put bracket here....
        random_labels.append([skip_grams_id[i][1]])
        
        #get cooc
        #first check whether it exists...
        pair = skip_grams[i]  #e.g., ('banana', 'fruit)
        try:
            cooc = X_ik[pair]
        except:
            cooc = 1 #label smoothing
            
        random_coocs.append([math.log(cooc)])  #1. why log, #2, why bracket -> size ==> (, 1)  #my neural network expects (, 1)
        
        #get weighting
        weighting = weighting_dic[pair]  #why not use try....maybe it does not exist....
        random_weighting.append(weighting)

        
    return np.array(random_inputs), np.array(random_labels), np.array(random_coocs), np.array(random_weighting)
    

<b>Test Function</b>

In [49]:
input, target, cooc, weightin = random_batch_gloVe(5, corpus_tokenized, gloVes, X_ik, weighting_dic)

In [50]:
print("Input sample:", input)
print(f"Input shape: {input.shape}", end="\n\n")
print(f"Label sample: {label=}")
print("Label shape:", label.shape)

Input sample: [[3679]
 [3899]
 [4080]
 [2613]
 [4806]]
Input shape: (5, 1)

Label sample: label=array([[2195, 2233, 4533, 1851],
       [7171, 6596, 2011, 3620],
       [1111, 3512, 2103, 2825],
       [7229, 4533, 2711, 4273],
       [ 510, 3494, 1006, 6999]])
Label shape: (5, 4)


In [51]:
input, target, cooc, weightin

(array([[3679],
        [3899],
        [4080],
        [2613],
        [4806]]),
 array([[3950],
        [4244],
        [4554],
        [5966],
        [5387]]),
 array([[0.69314718],
        [0.69314718],
        [0.69314718],
        [0.69314718],
        [0.69314718]]),
 array([0.05318296, 0.05318296, 0.05318296, 0.05318296, 0.05318296]))

### create helper function for training the model 

In [52]:
def prepare_sequence(seq, word2index):
    #map(function, list of something)
    #map will look at each of element in this list, and apply this function
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return torch.LongTensor(idxs)

In [53]:
import random
#you don't want to pick samples = targets, basically negative samples
#k = number of negative samples - how many? they found 10 is the best
#will be run during training
#after random_batch, 
def negative_sampling(targets, unigram_table, k):
    #targets is already in id.....
    #but the unigram_table is in word....
    #1. get the batch size of this targets
    batch_size = targets.shape[0]
    neg_samples = []
    #2. for each batch
    for i in range(batch_size):
        #randomly pick k negative words from unigram_table
        target_index = targets[i].item()  #looping each of the batch....
        nsample = []
        while len(nsample) < k:
            neg = random.choice(unigram_table)
            #if this word == target, skip this word
            if word2index[neg] == target_index:
                continue
            nsample.append(neg)
        #append this word to some list
        neg_samples.append(prepare_sequence(nsample, word2index).reshape(1, -1))  #tensor[], tensor[]
    return torch.cat(neg_samples)  #tensor[[], []]

<b>Test Functions</b>

In [54]:
input_batch, label_batch = random_batch(2, corpus_tokenized, skipgrams)

input_batch, label_batch

(array([[3052],
        [ 148]]),
 array([[3239],
        [3152]]))

In [55]:
input_batch = torch.LongTensor(input_batch)
label_batch = torch.LongTensor(label_batch)

In [56]:
num_neg = 5  #in the real code, we gonna use 10 (like in the paper)
neg_samples = negative_sampling(label_batch, unigram_table, num_neg)

In [57]:
neg_samples.shape

torch.Size([2, 5])

## 3) Model

### Skip-gram

$$J(\theta) = -\frac{1}{T}\sum_{t=1}^{T}\sum_{\substack{-m \leq j \leq m \\ j \neq 0}}\log P(w_{t+j} | w_t; \theta)$$

where $P(w_{t+j} | w_t; \theta) = $

$$P(o|c)=\frac{\exp(\mathbf{u_o^{\top}v_c})}{\sum_{w=1}^V\exp(\mathbf{u_w^{\top}v_c})}$$

where $o$ is the outside words and $c$ is the center word

In [58]:
#the model will accept three vectors - u_o, v_c, u_w
#u_o - vector for outside words
#v_c - vector for center word
#u_w - vectors of all vocabs

class Skipgram(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(Skipgram, self).__init__()
        self.embedding_center_word  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside_word = nn.Embedding(voc_size, emb_size)
    
    def forward(self, center_word, outside_word, all_vocabs):
        #center_word, outside_word: (batch_size, 1)
        #all_vocabs: (batch_size, voc_size)
        
        #convert them into embedding
        center_word_embed  = self.embedding_center_word(center_word)     #(batch_size, 1, emb_size)
        outside_word_embed = self.embedding_outside_word(outside_word)   #(batch_size, 1, emb_size)
        all_vocabs_embed   = self.embedding_outside_word(all_vocabs)     #(batch_size, voc_size, emb_size)
        
        #bmm is basically @ or .dot , but across batches (i.e., ignore the batch dimension)
        top_term = outside_word_embed.bmm(center_word_embed.transpose(1, 2)).squeeze(2)
        #(batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) ===> (batch_size, 1)
        
        top_term_exp = torch.exp(top_term)  #exp(uo vc)
        #(batch_size, 1)
        
        lower_term = all_vocabs_embed.bmm(center_word_embed.transpose(1, 2)).squeeze(2)
         #(batch_size, voc_size, emb_size) @ (batch_size, emb_size, 1) = (batch_size, voc_size, 1) = (batch_size, voc_size)
         
        lower_term_sum = torch.sum(torch.exp(lower_term), 1) #sum exp(uw vc)
        #(batch_size, 1)
        
        loss_fn = -torch.mean(torch.log(top_term_exp / lower_term_sum))
        #(batch_size, 1) / (batch_size, 1) ==mean==> scalar
        
        return loss_fn

In [59]:
#preparing all_vocabs

batch_size = 5

voc_size = len(vocabs)
voc_size

def prepare_sequence(seq, word2index):
    #map(function, list of something)
    #map will look at each of element in this list, and apply this function
    idxs = list(map(lambda w: word2index[w] if word2index.get(w) is not None else word2index["<UNK>"], seq))
    return torch.LongTensor(idxs)

all_vocabs = prepare_sequence(list(vocabs), word2index).expand(batch_size, voc_size)
all_vocabs.shape

torch.Size([5, 7324])

<b>Test Model</b>

In [60]:
input, label = random_batch(batch_size, corpus_tokenized, skipgrams)

In [61]:
input, label

(array([[3101],
        [3593],
        [4506],
        [2081],
        [6785]]),
 array([[5232],
        [5937],
        [5268],
        [5680],
        [3376]]))

In [62]:
emb_size = 2
model = Skipgram(voc_size, emb_size)

In [63]:
input_tensor = torch.LongTensor(input)  
label_tensor = torch.LongTensor(label) 

In [64]:
loss = model(input_tensor, label_tensor, all_vocabs)

In [65]:
loss

tensor(9.8243, grad_fn=<NegBackward0>)

### Skip-gram with negative sampling

$$\mathbf{J}_{\text{neg-sample}}(\mathbf{v}_c,o,\mathbf{U})=-\log(\sigma(\mathbf{u}_o^T\mathbf{v}_c))-\sum_{k=1}^K\log(\sigma(-\mathbf{u}_k^T\mathbf{v}_c))$$

In [66]:
class SkipgramNeg(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(SkipgramNeg, self).__init__()
        self.embedding_center_word  = nn.Embedding(voc_size, emb_size)
        self.embedding_outside_word = nn.Embedding(voc_size, emb_size)
        self.logsigmoid = nn.LogSigmoid()
        
    def forward(self, center_words, outside_words, negative_words):
        #center_words, outside_words: (batch_size, 1)
        #negative_words:  (batch_size, k)
        
        center_embed  = self.embedding_center_word(center_words)    #(batch_size, 1, emb_size)
        outside_embed = self.embedding_outside_word(outside_words)  #(batch_size, 1, emb_size)
        neg_embed     = self.embedding_outside_word(negative_words) #(batch_size, k, emb_size)
        
        uovc          =  outside_embed.bmm(center_embed.transpose(1, 2)).squeeze(2)  #(batch_size, 1)
        ukvc          = -neg_embed.bmm(center_embed.transpose(1, 2)).squeeze(2)  #(batch_size, k)
        ukvc_sum      =  torch.sum(ukvc, 1).view(-1, 1) #(batch_size, 1)
        
        loss = self.logsigmoid(uovc) + self.logsigmoid(ukvc_sum)  #(batch_size, 1) + (batch_size, 1)
                
        return -torch.mean(loss)  #scalar, loss should be scalar, to call backward()

<b>Test Model</b>

In [67]:
input, label = random_batch(batch_size, corpus_tokenized, skipgrams)
input_tensor = torch.LongTensor(input)  
label_tensor = torch.LongTensor(label)

In [68]:
emb_size = 2
voc_size = len(vocabs)
model = SkipgramNeg(voc_size, emb_size)

In [69]:
neg_tensor = negative_sampling(label_tensor, unigram_table, 5)

In [70]:
input_tensor.shape, label_tensor.shape#, neg_tensor.shape

(torch.Size([5, 1]), torch.Size([5, 1]))

In [71]:
loss = model(input_tensor, label_tensor, neg_tensor)

In [72]:
loss

tensor(2.1893, grad_fn=<NegBackward0>)

### CBOW

In [73]:
#the model will accept three vectors - u_c, u_j, v_mean
#u_c - vector for center word from the output word matrix
#u_j - vectors for all vocab from the output word matrix
#v_mean - mean of the vectors of context words from the input word matrix 

class CBOW(nn.Module):
    
    def __init__(self, voc_size, emb_size):
        super(CBOW, self).__init__()
        self.emb_size = emb_size
        self.input_word  = nn.Embedding(voc_size, emb_size) #v
        self.output_word = nn.Embedding(voc_size, emb_size) #u
    
    def forward(self, center_word, outside_word, all_vocabs):
        #center_word: (batch_size, 1)
        #context_words: (batch_size, window_size * 2)
        #all_vocabs: (batch_size, voc_size)
        batch_size = center_word.shape[0]
        
        #convert them into embedding
        center_word_embed  = self.output_word(center_word)   #(batch_size, 1, emb_size)
        outside_word_embed = self.input_word(outside_word)   #(batch_size, window_size * 2, emb_size)
        all_vocabs_embed   = self.output_word(all_vocabs)    #(batch_size, voc_size, emb_size)        
        
        # mean of input word embeddings
        v_mean = torch.sum(outside_word_embed, 1) / len(outside_word) #(batch_size, emb_size)
        
        ucv = center_word_embed.bmm(v_mean.reshape(batch_size, 1, self.emb_size).transpose(1, 2)).squeeze()
        #(batch_size, 1, emb_size) @ (batch_size, emb_size, 1) = (batch_size, 1, 1) ==> (batch_size, 1)
        
        ujv = all_vocabs_embed.bmm(v_mean.reshape(batch_size, 1, self.emb_size).transpose(1, 2)).squeeze(2)
        #(batch_size, voc_size, emb_size) @ (batch_size, emb_size, 1) = (batch_size, voc_size)
        
        ujv_log_exp = torch.log(torch.exp(ujv))
        # (batch_size, voc_size) -> (batch_size,)
        
        loss_fn = - ucv + torch.sum(ujv_log_exp, 1)
        # - (batch_size, 1) + (batch_size, 1) = (batch_size, 1)
        
        return torch.mean(loss_fn) # scaler for back-propagation

<b>Test Model</b>

In [74]:
all_vocabs = prepare_sequence(list(vocabs), word2index).expand(5, voc_size)
model = CBOW(voc_size, emb_size=2)

In [75]:
input, label = random_batch_cbow(5, corpus_tokenized, skipgrams_CBOW)
input_tensor = torch.LongTensor(input)  
label_tensor = torch.LongTensor(label)
input_tensor.shape, label_tensor.shape

(torch.Size([5, 1]), torch.Size([5, 4]))

In [76]:
input_tensor[0], label_tensor[0]

(tensor([4840]), tensor([5644, 5665,  944, 3939]))

In [77]:
model(input_tensor, label_tensor, all_vocabs)

tensor(-0.3047, grad_fn=<MeanBackward0>)

### GloVe

In [78]:
class GloVe(nn.Module):
    
    def __init__(self, vocab_size,embed_size):
        super(GloVe,self).__init__()
        self.embedding_v = nn.Embedding(vocab_size, embed_size) # center embedding
        self.embedding_u = nn.Embedding(vocab_size, embed_size) # out embedding
        
        self.v_bias = nn.Embedding(vocab_size, 1)
        self.u_bias = nn.Embedding(vocab_size, 1)
        
    def forward(self, center_words, target_words, coocs, weighting):
        center_embeds = self.embedding_v(center_words) # [batch_size, 1, emb_size]
        target_embeds = self.embedding_u(target_words) # [batch_size, 1, emb_size]
        
        center_bias = self.v_bias(center_words).squeeze(1)
        target_bias = self.u_bias(target_words).squeeze(1)
        
        inner_product = target_embeds.bmm(center_embeds.transpose(1, 2)).squeeze(2)
        #[batch_size, 1, emb_size] @ [batch_size, emb_size, 1] = [batch_size, 1, 1] = [batch_size, 1]
        
        #note that coocs already got log
        loss = weighting*torch.pow(inner_product +center_bias + target_bias - coocs, 2)
        
        return torch.sum(loss)

<b>Test Model</b>

In [79]:
input, target, cooc, weightin = random_batch_gloVe(5, corpus_tokenized, gloVes, X_ik, weighting_dic)

In [80]:
emb_size = 2
voc_size = len(vocabs)
model = GloVe(voc_size, emb_size)

In [81]:
input_batch    = torch.LongTensor(input)
target_batch   = torch.LongTensor(target)
cooc_batch     = torch.FloatTensor(cooc)
weightin_batch = torch.FloatTensor(weightin)

In [82]:
loss = model(input_batch, target_batch, cooc_batch, weightin_batch)

In [83]:
loss

tensor(7.5011, grad_fn=<SumBackward0>)

## 4) Training Model

In [84]:
import time

In [85]:
num_epochs = 5000

### Skip-gram

In [86]:
emb_size   = 50
model1     = Skipgram(voc_size, emb_size)
optimizer  = optim.Adam(model.parameters(), lr=0.001)
batch_size = 25

In [87]:
all_vocabs = prepare_sequence(list(vocabs), word2index).expand(batch_size, voc_size)
all_vocabs.shape

torch.Size([25, 7324])

In [88]:
train_start_time = time.time()

#for epoch
for epoch in range(num_epochs):
    start_time = time.time()
    #get random batch
    input_batch, label_batch = random_batch(batch_size, corpus_tokenized, skipgrams)
    input_batch = torch.LongTensor(input_batch)
    label_batch = torch.LongTensor(label_batch)
    
    # print(input_batch.shape, label_batch.shape, all_vocabs.shape)
    
    #loss = model
    loss = model1(input_batch, label_batch, all_vocabs)
    
    #backpropagate
    loss.backward()
    
    #update alpha
    optimizer.step()
    
    total_time = time.time() - start_time
    #print epoch loss
    if (epoch + 1) % 250 == 0:
        print(f"Epoch {epoch+1} | Loss: {loss:.6f} | Time: {total_time}")
        
total_training_time = time.time() - train_start_time
print("Total trainig time:", total_training_time)

Epoch 250 | Loss: 26.595457 | Time: 0.027086973190307617
Epoch 500 | Loss: 25.581800 | Time: 0.029000520706176758
Epoch 750 | Loss: 29.168341 | Time: 0.029961585998535156
Epoch 1000 | Loss: 27.501553 | Time: 0.030039072036743164
Epoch 1250 | Loss: 29.165424 | Time: 0.03065776824951172
Epoch 1500 | Loss: 27.259912 | Time: 0.02876758575439453
Epoch 1750 | Loss: 28.714096 | Time: 0.02934885025024414
Epoch 2000 | Loss: 29.096113 | Time: 0.028376102447509766
Epoch 2250 | Loss: 27.671209 | Time: 0.026983261108398438
Epoch 2500 | Loss: 27.055780 | Time: 0.02785968780517578
Epoch 2750 | Loss: 28.731546 | Time: 0.03099226951599121
Epoch 3000 | Loss: 26.388475 | Time: 0.0287477970123291
Epoch 3250 | Loss: 28.094547 | Time: 0.03195595741271973
Epoch 3500 | Loss: 27.826181 | Time: 0.030002832412719727
Epoch 3750 | Loss: 26.896601 | Time: 0.028470277786254883
Epoch 4000 | Loss: 26.892872 | Time: 0.03127622604370117
Epoch 4250 | Loss: 30.907562 | Time: 0.026211261749267578
Epoch 4500 | Loss: 27.2441

### Skip-gram with Negative Sampling

In [89]:
emb_size  = 50
model2 = SkipgramNeg(voc_size, emb_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
batch_size = 25

In [90]:
train_start_time = time.time()

#for epoch
for epoch in range(num_epochs):
    start_time = time.time()
    
    #get random batch
    input_batch, label_batch = random_batch(batch_size, corpus_tokenized, skipgrams)
    input_batch = torch.LongTensor(input_batch)
    label_batch = torch.LongTensor(label_batch)
    neg_batch   = negative_sampling(label_batch, unigram_table, 5)    
    
    #loss = model
    loss = model2(input_batch, label_batch, neg_batch)
    
    #backpropagate
    loss.backward()
    
    #update alpha
    optimizer.step()
    
    total_time = time.time() - start_time
    #print epoch loss
    if (epoch + 1) % 250 == 0:
        print(f"Epoch {epoch+1} | Loss: {loss:.6f} | Time: {total_time}")
        
total_training_time = time.time() - train_start_time
print("Total trainig time:", total_training_time)

Epoch 250 | Loss: 8.205234 | Time: 0.007005929946899414
Epoch 500 | Loss: 6.990298 | Time: 0.007039785385131836
Epoch 750 | Loss: 6.567665 | Time: 0.007110118865966797
Epoch 1000 | Loss: 8.909377 | Time: 0.008394479751586914
Epoch 1250 | Loss: 6.478815 | Time: 0.008036613464355469
Epoch 1500 | Loss: 8.917675 | Time: 0.007954835891723633
Epoch 1750 | Loss: 9.195609 | Time: 0.00767207145690918
Epoch 2000 | Loss: 7.721918 | Time: 0.008297204971313477
Epoch 2250 | Loss: 9.909580 | Time: 0.007026195526123047
Epoch 2500 | Loss: 8.253580 | Time: 0.008241891860961914
Epoch 2750 | Loss: 10.001043 | Time: 0.00700831413269043
Epoch 3000 | Loss: 9.415213 | Time: 0.009051084518432617
Epoch 3250 | Loss: 11.735939 | Time: 0.005998373031616211
Epoch 3500 | Loss: 9.457308 | Time: 0.00823354721069336
Epoch 3750 | Loss: 11.514767 | Time: 0.00706028938293457
Epoch 4000 | Loss: 11.432185 | Time: 0.008025884628295898
Epoch 4250 | Loss: 9.038859 | Time: 0.007652997970581055
Epoch 4500 | Loss: 8.878803 | Time

### CBOW

In [91]:
emb_size  = 50
model3 = CBOW(voc_size, emb_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
batch_size = 25

In [92]:
all_vocabs = prepare_sequence(list(vocabs), word2index).expand(batch_size, voc_size)
all_vocabs.shape

torch.Size([25, 7324])

In [93]:
train_start_time = time.time()

#for epoch
for epoch in range(num_epochs):
    start_time = time.time()
    #get random batch
    input_batch, label_batch = random_batch_cbow(batch_size, corpus_tokenized, skipgrams_CBOW)
    input_batch = torch.LongTensor(input_batch)
    label_batch = torch.LongTensor(label_batch)
    
    #loss = model
    loss = model3(input_batch, label_batch, all_vocabs)
    
    #backpropagate
    loss.backward()
    
    #update alpha
    optimizer.step()
    
    total_time = time.time() - start_time
    #print epoch loss
    if (epoch + 1) % 250 == 0:
        print(f"Epoch {epoch+1} | Loss: {loss:.6f} | Time: {total_time}")
        
total_training_time = time.time() - train_start_time
print("Total trainig time:", total_training_time)

Epoch 250 | Loss: 0.519273 | Time: 0.02568840980529785
Epoch 500 | Loss: 9.708236 | Time: 0.023740291595458984
Epoch 750 | Loss: -1.130049 | Time: 0.030622243881225586
Epoch 1000 | Loss: 9.584751 | Time: 0.028729677200317383
Epoch 1250 | Loss: -29.188667 | Time: 0.02927851676940918
Epoch 1500 | Loss: 3.678998 | Time: 0.029614686965942383
Epoch 1750 | Loss: -1.195722 | Time: 0.027408361434936523
Epoch 2000 | Loss: -1.211780 | Time: 0.030165672302246094
Epoch 2250 | Loss: -1.421137 | Time: 0.029845237731933594
Epoch 2500 | Loss: 0.652337 | Time: 0.02757096290588379
Epoch 2750 | Loss: -5.580129 | Time: 0.029867887496948242
Epoch 3000 | Loss: -2.801577 | Time: 0.027363061904907227
Epoch 3250 | Loss: 5.922264 | Time: 0.027045249938964844
Epoch 3500 | Loss: -3.870833 | Time: 0.029249906539916992
Epoch 3750 | Loss: -1.101122 | Time: 0.028268814086914062
Epoch 4000 | Loss: -2.036714 | Time: 0.027559995651245117
Epoch 4250 | Loss: -2.027371 | Time: 0.02759718894958496
Epoch 4500 | Loss: 0.25004

### GloVe

In [94]:
emb_size  = 50
model4 = GloVe(voc_size, emb_size)
optimizer = optim.Adam(model.parameters(), lr=0.001)
batch_size = 25

In [95]:
train_start_time = time.time()

#for epoch
for epoch in range(num_epochs):
    start_time = time.time()
    
    #get random batch
    input, target, cooc, weightin = random_batch_gloVe(batch_size, corpus_tokenized, gloVes, X_ik, weighting_dic)
    input_batch    = torch.LongTensor(input)
    target_batch   = torch.LongTensor(target)
    cooc_batch     = torch.FloatTensor(cooc)
    weightin_batch = torch.FloatTensor(weightin)
        
    #loss = model
    loss = model4(input_batch, target_batch, cooc_batch, weightin_batch)
    
    #backpropagate
    loss.backward()
    
    #update alpha
    optimizer.step()
    
    total_time = time.time() - start_time
    
    #print epoch loss
    if (epoch + 1) % 250 == 0:
        print(f"Epoch {epoch+1} | Loss: {loss:.6f} | Time: {total_time}")
        
total_training_time = time.time() - train_start_time
print("Total trainig time:", total_training_time)

Epoch 250 | Loss: 1997.681274 | Time: 0.014145135879516602
Epoch 500 | Loss: 1937.284180 | Time: 0.010567426681518555
Epoch 750 | Loss: 1650.354126 | Time: 0.008912086486816406
Epoch 1000 | Loss: 2416.785156 | Time: 0.008843660354614258
Epoch 1250 | Loss: 2472.373535 | Time: 0.010133028030395508
Epoch 1500 | Loss: 2394.305908 | Time: 0.009006500244140625
Epoch 1750 | Loss: 1660.030029 | Time: 0.008988618850708008
Epoch 2000 | Loss: 2496.681152 | Time: 0.008260965347290039
Epoch 2250 | Loss: 1494.736694 | Time: 0.009389162063598633
Epoch 2500 | Loss: 2282.000000 | Time: 0.008575201034545898
Epoch 2750 | Loss: 1788.161621 | Time: 0.009713172912597656
Epoch 3000 | Loss: 2007.218262 | Time: 0.00727391242980957
Epoch 3250 | Loss: 921.957581 | Time: 0.008416175842285156
Epoch 3500 | Loss: 2381.812012 | Time: 0.009227991104125977
Epoch 3750 | Loss: 1912.392090 | Time: 0.008805274963378906
Epoch 4000 | Loss: 2906.250732 | Time: 0.008000850677490234
Epoch 4250 | Loss: 2709.157715 | Time: 0.0080

## 5) Evaluating Embeddings

### Semantic and syntatic Accuracy
trying every word in the vocabulary to find the word that has the highest cosine similarity with the vector B - A + C. If the word is in the semantic category of the question, then the accuracy is increased by 1. 




all these models were trained using the first 2000 sentences from the brown corpus category. (to reduce the vocab size) doesn't seems to indentify the missing term correctly for both semantic and syntactic parts. This may be due to limited corpus size and vocabulary. Due to this, another accuracy based on how the correct missing term is ranked based on similarity is used. The average was taken from very analogy task questions for both semantic and syntactic parts.

In [96]:
def get_embed(word, model):
    try:
        index = word2index[word]
    except:
        index = word2index['<UNK>']
    
    word = torch.LongTensor([index])

    center_embed  = model.embedding_center_word(word)
    outside_embed = model.embedding_outside_word(word)
    
    embed = (center_embed + outside_embed) / 2
    
    return embed
    # return  embed[0][0].item(), embed[0][1].item()

In [97]:
get_embed("man", model1).shape

torch.Size([1, 50])

In [98]:
def get_embed_CBOW(word, model):
    try:
        index = word2index[word]
    except:
        index = word2index['<UNK>']
    
    word = torch.LongTensor([index])

    center_embed  = model.input_word(word)
    outside_embed = model.output_word(word)
    
    embed = (center_embed + outside_embed) / 2
    
    return embed
    # return  embed[0][0].item(), embed[0][1].item()

In [99]:
get_embed_CBOW("man", model3).shape

torch.Size([1, 50])

In [100]:
def get_embed_GloVe(word, model):
    id_tensor = torch.LongTensor([word2index[word]])
    v_embed = model.embedding_v(id_tensor)
    u_embed = model.embedding_u(id_tensor) 
    word_embed = (v_embed + u_embed) / 2 
    
    return word_embed

    # x, y = word_embed[0][0].item(), word_embed[0][1].item()
    # return x, y

In [101]:
get_embed_GloVe("man", model4).shape

torch.Size([1, 50])

In [102]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a, b):
    cos_sim = dot(a, b)/(norm(a)*norm(b))
    return cos_sim

### testing the model using questions-words.txt dataset 
### modifying the dataset to fit the model 

In [103]:
#open and load the questions-words.txt file
with open('questions-words.txt', 'r') as f:
    questions = f.readlines()

#remove the first 5 lines
questions = questions[5:]

#remove the last 2 lines
questions = questions[:-2]

#remove the new line character
questions = [q.strip() for q in questions]

#remove the empty lines
questions = [q for q in questions if q]

#split the questions into 4 parts
questions = [q.split(' ') for q in questions]

#remove the colon
questions = [q[1:] for q in questions]

#remove the last word
questions = [q[:-1] for q in questions]




In [104]:
testDataset = {}
key         = None
value       = []
with open('questions-words.txt') as f:
    for line in f.readlines():
        if line[0] == ":":
            if key != None:
                testDataset[key] = value
            key   = line.strip()
            value = []
            continue
        value.append(line.strip().lower())
    testDataset[key] = value

In [105]:
len(testDataset)

14

In [106]:
testDataset.keys()

dict_keys([': capital-common-countries', ': capital-world', ': currency', ': city-in-state', ': family', ': gram1-adjective-to-adverb', ': gram2-opposite', ': gram3-comparative', ': gram4-superlative', ': gram5-present-participle', ': gram6-nationality-adjective', ': gram7-past-tense', ': gram8-plural', ': gram9-plural-verbs'])

In [107]:
testDataset[': capital-common-countries'][:10]

['athens greece baghdad iraq',
 'athens greece bangkok thailand',
 'athens greece beijing china',
 'athens greece berlin germany',
 'athens greece bern switzerland',
 'athens greece cairo egypt',
 'athens greece canberra australia',
 'athens greece hanoi vietnam',
 'athens greece havana cuba',
 'athens greece helsinki finland']

In [108]:
semantic  = [': capital-common-countries', ': capital-world', ': currency', ': family']
syntactic = [': gram1-adjective-to-adverb', ': gram2-opposite', ': gram3-comparative', ': gram4-superlative', 
             ': gram5-present-participle', ': gram6-nationality-adjective', ': gram7-past-tense', ': gram8-plural', 
             ': gram9-plural-verbs']

### okay now moving on to semantic accuracy

In [109]:
def semantic_syntatic_eval(model, testDict, CBOWMode = False, GloVeMode = False):
    total_corr = 0
    pairs_used = 0
    acc_sum    = 0
    
    for key in testDict:
        pairs = testDataset[key]
        # print(pairs[0].split(" "))
        for pair in pairs:
            pair_tokenized = pair.split(" ")
            word_a, word_b, word_c, word_d = pair_tokenized  
            if word_a not in word2index or word_b not in word2index or word_c not in word2index or word_d not in word2index:
                continue
                
            word_d_index = vocabs.index(word_d)
            
            if GloVeMode:
                a_embedding = get_embed_GloVe(word_a, model) 
                b_embedding = get_embed_GloVe(word_b, model)
                c_embedding = get_embed_GloVe(word_c, model)
            elif CBOWMode:
                a_embedding = get_embed_CBOW(word_a, model) 
                b_embedding = get_embed_CBOW(word_b, model)
                c_embedding = get_embed_CBOW(word_c, model)
            else:
                a_embedding = get_embed(word_a, model) 
                b_embedding = get_embed(word_b, model)
                c_embedding = get_embed(word_c, model)
            
            AminusBplusC = (b_embedding - a_embedding + c_embedding).squeeze()
            
            count = -1
            similarity_arr = [0] * len(vocabs)
            for vocab in vocabs:
                count += 1
                if vocab in pair_tokenized[:3]:
                    continue
                
                if GloVeMode:
                    current = get_embed_GloVe(vocab, model).squeeze()
                elif CBOWMode:
                    current = get_embed_CBOW(vocab, model).squeeze()
                else:
                    current = get_embed(vocab, model).squeeze()
                similarity_arr[count] = cos_sim(AminusBplusC.detach().numpy(), current.detach().numpy())
            
            similarity_arr_sorted_index = np.argsort(similarity_arr)
            rank                        = np.where(similarity_arr_sorted_index == word_d_index)[0][0]
            acc_sum += (rank + 1) / len(vocabs)
            
            pairs_used += 1
            predicted_word = np.argmax(similarity_arr)
            if predicted_word == word_d:
                total_corr+= 1
                
    total_acc = total_corr / pairs_used
    avg_acc = acc_sum / pairs_used
    return total_acc, total_corr, avg_acc, pairs_used

##### Skip-gram

In [110]:
total_acc, total_corr, avg_acc, pairs_used = semantic_syntatic_eval(model1, semantic)

In [111]:
print(f"Overall Accuracy: {total_acc}%")
print(f"Average accuracy according to the ranking: {round(avg_acc, 2)}%")
print(f"Total pairs used:", pairs_used)

Overall Accuracy: 0.0%
Average accuracy according to the ranking: 0.49%
Total pairs used: 113


##### Skip-gram with negative sampling

In [112]:
total_acc_neg, total_corr_neg, avg_acc_neg, pairs_used_neg = semantic_syntatic_eval(model2, semantic)

In [113]:
print(f"Overall Accuracy: {total_acc_neg}%")
print(f"Average accuracy according to the ranking: {round(avg_acc_neg, 2)}%")
print(f"Total pairs used:", pairs_used_neg)

Overall Accuracy: 0.0%
Average accuracy according to the ranking: 0.44%
Total pairs used: 113


##### CBOW

In [114]:
total_acc_cbow, total_corr_cbow, avg_acc_cbow, pairs_used_cbow = semantic_syntatic_eval(model3, semantic, CBOWMode=True)

In [115]:
print(f"Overall Accuracy: {total_acc_cbow}%")
print(f"Average accuracy according to the ranking: {round(avg_acc_cbow, 2)}%")
print(f"Total pairs used:", pairs_used_cbow)

Overall Accuracy: 0.0%
Average accuracy according to the ranking: 0.5%
Total pairs used: 113


##### GloVe

In [116]:
total_acc_gloVe, total_corr_gloVe, avg_acc_gloVe, pairs_used_gloVe = semantic_syntatic_eval(model4, semantic, GloVeMode=True)

In [117]:
print(f"Overall Accuracy: {total_acc_gloVe}%")
print(f"Average accuracy according to the ranking: {round(avg_acc_gloVe, 2)}%")
print(f"Total pairs used:", pairs_used_gloVe)

Overall Accuracy: 0.0%
Average accuracy according to the ranking: 0.45%
Total pairs used: 113


### and now Syntatic 

##### Skip-gram

In [118]:
total_acc2, total_corr2, avg_acc2, pairs_used2 = semantic_syntatic_eval(model1, syntactic)

In [119]:
print(f"Overall Accuracy: {total_acc2}%")
print(f"Average accuracy according to the ranking: {round(avg_acc2, 2)}%")
print(f"Total pairs used:", pairs_used2)

Overall Accuracy: 0.0%
Average accuracy according to the ranking: 0.5%
Total pairs used: 1108


##### Skip-gram with negative sampling

In [120]:
total_acc_neg2, total_corr_neg2, avg_acc_neg2, pairs_used_neg2 = semantic_syntatic_eval(model2, syntactic)

In [121]:
print(f"Overall Accuracy: {total_acc_neg2}%")
print(f"Average accuracy according to the ranking: {round(avg_acc_neg2, 2)}%")
print(f"Total pairs used:", pairs_used_neg2)

Overall Accuracy: 0.0%
Average accuracy according to the ranking: 0.47%
Total pairs used: 1108


##### CBOW

In [122]:
total_acc_cbow2, total_corr_cbow2, avg_acc_cbow2, pairs_used_cbow2 = semantic_syntatic_eval(model3, syntactic, CBOWMode=True)

In [123]:
print(f"Overall Accuracy: {total_acc_cbow2}%")
print(f"Average accuracy according to the ranking: {round(avg_acc_cbow2, 2)}%")
print(f"Total pairs used:", pairs_used_cbow2)

Overall Accuracy: 0.0%
Average accuracy according to the ranking: 0.51%
Total pairs used: 1108


##### GloVe

In [124]:
total_acc_gloVe2, total_corr_gloVe2, avg_acc_gloVe2, pairs_used_gloVe2 = semantic_syntatic_eval(model4, syntactic, GloVeMode=True)

In [125]:
print(f"Overall Accuracy: {total_acc_gloVe2}%")
print(f"Average accuracy according to the ranking: {round(avg_acc_gloVe2, 2)}%")
print(f"Total pairs used:", pairs_used_gloVe2)

Overall Accuracy: 0.0%
Average accuracy according to the ranking: 0.5%
Total pairs used: 1108


### accuracy results for semantic and syntatic parts

In [127]:
from tabulate import tabulate

In [128]:
total_pairs_semantic = 0
for key in semantic:
    total_pairs_semantic += len(testDataset[key])

In [129]:
print("Total pairs for semantic evaluation:", total_pairs_semantic)
print("Pairs used for semantic evaluation:", pairs_used)

Total pairs for semantic evaluation: 6402
Pairs used for semantic evaluation: 113


In [130]:
total_pairs_syntactic = 0
for key in syntactic:
    total_pairs_syntactic += len(testDataset[key])

In [131]:
print("Total pairs for syntactic evaluation:", total_pairs_syntactic)
print("Pairs used for syntactic evaluatio:", pairs_used2)

Total pairs for syntactic evaluation: 10675
Pairs used for syntactic evaluatio: 1108


### Total Accuracy (%)

In [132]:
head    = ["Model", "Semantic", "Syntatic"]
content = [
    ["Skip-gram", total_acc, total_acc2],
    ["Skip-gram with negative sampling", total_acc_neg, total_acc_neg2],
    ["CBOW", total_acc_cbow, total_acc_cbow2],
    ["GloVe", total_acc_gloVe, total_acc_gloVe2]
] 

In [133]:
print(tabulate(content, headers=head, tablefmt="grid"))

+----------------------------------+------------+------------+
| Model                            |   Semantic |   Syntatic |
+==================================+============+============+
| Skip-gram                        |          0 |          0 |
+----------------------------------+------------+------------+
| Skip-gram with negative sampling |          0 |          0 |
+----------------------------------+------------+------------+
| CBOW                             |          0 |          0 |
+----------------------------------+------------+------------+
| GloVe                            |          0 |          0 |
+----------------------------------+------------+------------+


### Average Accuracy (%) based on Ranking

In [134]:
head    = ["Model", "Semantic", "Syntatic"]
content = [
    ["Skip-gram", avg_acc, avg_acc2],
    ["Skip-gram with negative sampling", avg_acc_neg, avg_acc_neg2],
    ["CBOW", avg_acc_cbow, avg_acc_cbow2],
    ["GloVe", avg_acc_gloVe, avg_acc_gloVe2]
] 

In [135]:
print(tabulate(content, headers=head, tablefmt="grid"))

+----------------------------------+------------+------------+
| Model                            |   Semantic |   Syntatic |
+==================================+============+============+
| Skip-gram                        |   0.491664 |   0.499768 |
+----------------------------------+------------+------------+
| Skip-gram with negative sampling |   0.443425 |   0.472926 |
+----------------------------------+------------+------------+
| CBOW                             |   0.502589 |   0.505397 |
+----------------------------------+------------+------------+
| GloVe                            |   0.448496 |   0.502787 |
+----------------------------------+------------+------------+


### finally let's check the correlation of the similarity given by the trained models with the similarity score given by the judge for word pairs similarity using wordsim353_sim_rel dataset 

In [144]:
from scipy import stats

### Load Similarity Dataset

In [136]:
with open('wordsim353_sim_rel/wordsim_relatedness_goldstandard.txt') as f:
    relatedness_db = [line.strip().lower().split("\t") for line in f.readlines()]

In [137]:
relatedness_db[:10]

[['computer', 'keyboard', '7.62'],
 ['jerusalem', 'israel', '8.46'],
 ['planet', 'galaxy', '8.11'],
 ['canyon', 'landscape', '7.53'],
 ['opec', 'country', '5.63'],
 ['day', 'summer', '3.94'],
 ['day', 'dawn', '7.53'],
 ['country', 'citizen', '7.31'],
 ['planet', 'people', '5.75'],
 ['environment', 'ecology', '8.81']]

In [138]:
with open('wordsim353_sim_rel/wordsim_similarity_goldstandard.txt') as f:
    similarity_db = [line.strip().lower().split("\t") for line in f.readlines()]

In [139]:
similarity_db[:10]

[['tiger', 'cat', '7.35'],
 ['tiger', 'tiger', '10.00'],
 ['plane', 'car', '5.77'],
 ['train', 'car', '6.31'],
 ['television', 'radio', '6.77'],
 ['media', 'radio', '7.42'],
 ['bread', 'butter', '6.19'],
 ['cucumber', 'potato', '5.92'],
 ['doctor', 'nurse', '7.00'],
 ['professor', 'doctor', '6.62']]

In [145]:
def similarity_eval(dataset, model, CBOWMode = False, GloVeMode = False):
    valid_idx     = []
    cos_sim_arr   = []
    human_val_sim = []
    
    for idx, pair in enumerate(dataset):
        word1, word2, evalHuman = pair
        if word1 not in vocabs or word2 not in vocabs:
            continue
        
        if GloVeMode:
            word1_embedding = get_embed_GloVe(word1, model)
            word2_embedding = get_embed_GloVe(word2, model)
        elif CBOWMode:
            word1_embedding = get_embed_CBOW(word1, model)
            word2_embedding = get_embed_CBOW(word2, model)
        else:
            word1_embedding = get_embed(word1, model)
            word2_embedding = get_embed(word2, model)
        
        sim_eval = cos_sim(word1_embedding.detach().numpy().squeeze(), word2_embedding.detach().numpy().squeeze())
        valid_idx.append(idx)
        cos_sim_arr.append(sim_eval)
        human_val_sim.append(float(evalHuman))
        
        res = stats.spearmanr(cos_sim_arr, human_val_sim)
    
    return valid_idx, cos_sim_arr, human_val_sim, res

### now let's check the correlation of the similarity given by the trained models with the similarity score given by the judge for word pairs similarity using wordsim353_sim_rel dataset 

spearmans correlation coefficient is used to check the correlation of the similarity given by the trained models with the similarity score given by the judge for word pairs similarity 



##### Skip-gram

In [146]:
valid_idx, cos_sim_arr, human_val_sim, res = similarity_eval(similarity_db, model1)

In [147]:
print("Total samples:", len(similarity_db))
print("Samples used:", len(valid_idx))
print("Spearmanr correlation coefficient (rs):", round(res.correlation,4))
print("p-value:", round(res.pvalue,4))

Total samples: 203
Samples used: 78
Spearmanr correlation coefficient (rs): -0.0468
p-value: 0.6839


##### Skip-gram with negative sampling

In [148]:
valid_idx_neg, cos_sim_arr_neg, human_val_sim_neg, res_neg = similarity_eval(similarity_db, model2)

In [149]:
print("Total samples:", len(similarity_db))
print("Samples used:", len(valid_idx_neg))
print("Spearmanr correlation coefficient (rs):", round(res_neg.correlation,4))
print("p-value:", round(res_neg.pvalue,4))

Total samples: 203
Samples used: 78
Spearmanr correlation coefficient (rs): -0.0835
p-value: 0.4673


##### CBOW

In [150]:
valid_idx_CBOW, cos_sim_arr_CBOW, human_val_sim_CBOW, res_CBOW = similarity_eval(similarity_db, model2)

In [151]:
print("Total samples:", len(similarity_db))
print("Samples used:", len(valid_idx_CBOW))
print("Spearmanr correlation coefficient (rs)::", round(res_CBOW.correlation,4))
print("p-value of Spearmanr:", round(res_CBOW.pvalue,4))

Total samples: 203
Samples used: 78
Spearmanr correlation coefficient (rs):: -0.0835
p-value of Spearmanr: 0.4673


##### GloVe

In [152]:
valid_idx_gloVe, cos_sim_arr_gloVe, human_val_sim_gloVe, res_gloVe = similarity_eval(similarity_db, model1)

In [153]:
print("Total samples:", len(similarity_db))
print("Samples used:", len(valid_idx_gloVe))
print("Spearmanr correlation coefficient (rs)::", round(res_gloVe.correlation,4))
print("p-value:", round(res_gloVe.pvalue,4))

Total samples: 203
Samples used: 78
Spearmanr correlation coefficient (rs):: -0.0468
p-value: 0.6839


### correlation results for different models using wordsim353_sim_rel dataset 

### Spearman correlation coefficient with associated p-value for different models :








In [154]:
head    = ["Model", "rs", "p-value"]
content = [
    ["Skip-gram", round(res.correlation,4), round(res.pvalue,4)],
    ["Skip-gram with negative sampling", round(res_neg.correlation,4), round(res_neg.pvalue,4)],
    ["CBOW", round(res_CBOW.correlation,4), round(res_CBOW.pvalue,4)],
    ["GloVe", round(res_gloVe.correlation,4), round(res_gloVe.pvalue,4)]
] 

In [155]:
print(tabulate(content, headers=head, tablefmt="grid"))
print("Where rs is Spearman correlation coefficient")

+----------------------------------+---------+-----------+
| Model                            |      rs |   p-value |
+==================================+=========+===========+
| Skip-gram                        | -0.0468 |    0.6839 |
+----------------------------------+---------+-----------+
| Skip-gram with negative sampling | -0.0835 |    0.4673 |
+----------------------------------+---------+-----------+
| CBOW                             | -0.0835 |    0.4673 |
+----------------------------------+---------+-----------+
| GloVe                            | -0.0468 |    0.6839 |
+----------------------------------+---------+-----------+
Where rs is Spearman correlation coefficient


## 6) Conclusion and Discussion 

### ---semantic and syntatic accuracy results for different models--- 
in summary, the skip-gram with negative sampling model seems to perform the best in both semantic and syntatic parts. the skip-gram model seems to perform the best in syntatic part. the CBOW model seems to perform the best in semantic part. the GloVe model seems to perform the best in syntatic part. They all do not perform well in both semantic and syntatic parts. This may be due to limited corpus size and vocabulary. Due to this, another accuracy based on how the correct missing term is ranked based on similarity is used. The average was taken from very analogy task questions for both semantic and syntatic parts. 




### ---correlation results for different models using wordsim353_sim_rel dataset---

Also for the correlation of the similarity given by the trained models with the similarity score given by the judge for word pairs similarity using wordsim353_sim_rel dataset is not very high, either . This may be due to limited corpus size and vocabulary. Due to this, another accuracy based on how the correct missing term is ranked based on similarity is used. The average was taken from very analogy task questions for both semantic and syntatic parts. 